In [1]:
import numpy as np
import librosa, librosa.util
from sklearn.decomposition import NMF
from scipy.io.wavfile import write
from librosa.core import icqt
import librosa.display
import IPython.display

In [2]:
y ,sr = librosa.load('./resource/violin.wav')
C = librosa.cqt(y, sr=sr, hop_length=64, n_bins=84, bins_per_octave=12)
C_n = np.abs(C)

In [3]:
IPython.display.Audio(data=y, rate=sr)

In [4]:
n_components = 2
nmf_model = NMF(n_components=n_components, init='random', max_iter=200, alpha=0, l1_ratio=0.0)

# Use Examples Ref  
```python
wav = icqt(C, sr=sr, hop_length=64, res_type='fft')
write('after.wav', sr, wav)
IPython.display.Audio(data=y, rate=sr)
```

# Todos  
- [x] Check **librosa** official example of [harmonic/percussive separation](https://nbviewer.jupyter.org/github/librosa/librosa/blob/master/examples/LibROSA%20audio%20effects%20and%20playback.ipynb)  
- [ ] Check possible solutions to the bottleneck of icqt problem  
    - [ ] Manully add virtual part to the matrix  
    > Depending on how it would affect the icqt  
    - [ ] Try other resample models  

In [5]:
S_full, phase = librosa.magphase(C)
print(type(S_full[0][0]), type(phase[0][0]))

<class 'numpy.float64'> <class 'numpy.complex128'>


In [6]:
W = nmf_model.fit_transform(S_full)
H = nmf_model.components_

In [23]:
n_W = []
n_H = []
n_S = []
n_C = []
n_wav = []
for i in range(0, n_components):
    w = W[:,i:i+1]
    h = H[i:i+1, :]
    s = np.multiply(w, h)
    c = s * phase
    wav = icqt(c, sr=sr, hop_length=64)
    
    n_W.append(w)
    n_H.append(h)
    n_S.append(s)
    n_C.append(c)
    n_wav.append(wav)

In [22]:
print(type(W), type(H), W.shape, H.shape)
print(type(w), type(h), w.shape, h.shape)
print(type(s), s.shape)
print(type(c), c.shape)
print(wav)
print(type(n_wav[0]))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> (84, 2) (2, 1593)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (84, 1) (1, 1593)
<class 'numpy.ndarray'> (84, 1593)
<class 'numpy.ndarray'> (84, 1593)
[-0.09932379 -0.10519342 -0.10679824 ... -0.0509811  -0.06849765
 -0.0811259 ]
<class 'list'>


In [24]:
IPython.display.Audio(data=n_wav[0], rate=sr)

# Todos
[librosa.beat.tempo](https://librosa.github.io/librosa/generated/librosa.beat.tempo.html#librosa.beat.tempo) could be used to detect and estimate tempo.  
[librosa.onset.onset_detect](https://librosa.github.io/librosa/generated/librosa.onset.onset_detect.html#librosa.onset.onset_detect) could be used to detect onset.  
[librosa.decompose.decompose](https://librosa.github.io/librosa/generated/librosa.decompose.decompose.html#librosa.decompose.decompose) decompose a feature matrix with **scikit-learn** as well.  